## Scratch pad to test functions from parametric hale

Define the components list:

In [7]:
import numpy as np
import sharpy.utils.algebra as algebra
import cases.models_generator.gen_main as gm
import cases.models_generator.gen_utils as gu
import sharpy.utils.h5utils as h5utils
import os
import sharpy.sharpy_main


In [8]:
components=['fuselage', 'wing_r', 'winglet_r',
                              'wing_l', 'winglet_l', 'vertical_tail',
                              'horizontal_tail_right', 'horizontal_tail_left']

print(type(components))

bound_panels=8
# aeroelasticity parameters
main_ea = 0.3  # Wing elastic axis from LE as %
main_cg = 0.3  # Not sure about this input
sigma = 1.5

# other
c_ref = 1.0
# Wing Stiffness & mass
ea, ga = 1.5e7, 1e5
gj = 1.5e4
eiy = 3e4
eiz = 6e5
base_stiffness = np.diag([ea, ga, ga, sigma * gj, sigma * eiy, eiz])
stiffness = np.zeros((1, 6, 6))
stiffness[0] = base_stiffness
m_unit = 0.75
j_tors = 0.075
pos_cg_b = np.array([0., c_ref * (main_cg - main_ea), 0.])
m_chi_cg = algebra.skew(m_unit * pos_cg_b)
mass_wing = np.zeros((1, 6, 6))
mass_wing[0, :, :] = np.diag([m_unit, m_unit, m_unit,
                              j_tors, .5 * j_tors, .5 * j_tors])
mass_wing[0, :3, 3:] = m_chi_cg
mass_wing[0, 3:, :3] = -m_chi_cg

# Tail Stiffness and mass of the horizontal tail
ea_tail = 0.5
sigma_tail = 10  # Use a multiplication factor
m_unit_tail = 0.3
j_tors_tail = 0.08

mass_tail = np.zeros((1, 6, 6))
mass_tail[0, :, :] = np.diag([m_unit_tail,
                              m_unit_tail,
                              m_unit_tail,
                              j_tors_tail,
                              .5 * j_tors_tail,
                              .5 * j_tors_tail])
mass_tail[0, :3, 3:] = m_chi_cg
mass_tail[0, 3:, :3] = -m_chi_cg
# Fuselage Stiffness and mass
sigma_fuselage = 10
m_unit_fuselage = 0.2
j_tors_fuselage = 0.08
mass_fuselage = np.zeros((1, 6, 6))
mass_fuselage[0, :, :] = np.diag([m_unit_fuselage,
                                  m_unit_fuselage,
                                  m_unit_fuselage,
                                  j_tors_fuselage,
                                  .5 * j_tors_fuselage,
                                  .5 * j_tors_fuselage])
mass_fuselage[0, :3, 3:] = m_chi_cg
mass_fuselage[0, 3:, :3] = -m_chi_cg

# Lumped mass
n_lumped_mass = 1  # Number of lumped masses
lumped_mass_nodes = np.zeros((n_lumped_mass,), dtype=int)  # Maps lumped mass to nodes
lumped_mass = np.zeros((n_lumped_mass,))  # Array of lumped masses in kg
lumped_mass[0] = 50
lumped_mass_inertia = np.zeros((n_lumped_mass, 3, 3))  # 3x3 inertia to the previous masses
lumped_mass_position = np.zeros((n_lumped_mass, 3))  # Relative position to the belonging node in B FoR

g1c = dict()
g1c['fuselage'] = {'workflow': ['read_structure', 'read_aero'],
                   'geometry': {'length': 10,
                                'num_node': 11,
                                'direction': [1., 0., 0.],
                                'sweep': 0.,
                                'dihedral': 0.},
                   'fem': {'stiffness_db': stiffness,
                           'mass_db': mass_fuselage,
                           'frame_of_reference_delta': [0, 1., 0.],
                           'lumped_mass': lumped_mass,
                           'lumped_mass_nodes': lumped_mass_nodes,
                           'lumped_mass_inertia': lumped_mass_inertia,
                           'lumped_mass_position': lumped_mass_position}
                   }

g1c['wing_r'] = {'workflow': ['create_structure', 'create_aero'],
                 'geometry': {'length': 20.,
                              'num_node': 11,
                              'direction': [0., 1., 0.],
                              'sweep': 0. * np.pi / 180,
                              'dihedral': 0.},
                 'fem': {'stiffness_db': stiffness,
                         'mass_db': mass_wing,
                         'frame_of_reference_delta': [-1, 0., 0.]},
                 'aero': {'chord': [1., 1.],
                          'elastic_axis': 0.33,
                          'surface_m': bound_panels}
                 }
g1c['winglet_r'] = {'workflow': ['create_structure', 'create_aero'],
                    'geometry': {'length': 4,
                                 'num_node': 3,
                                 'direction': [0., 1., 0.],
                                 'sweep': 0. * np.pi / 180,
                                 'dihedral': 20. * np.pi / 180},
                    'fem': {'stiffness_db': stiffness,
                            'mass_db': mass_wing,
                            'frame_of_reference_delta': [-1, 0., 0.]},
                    'aero': {'chord': [1., 1.],
                             'elastic_axis': 0.33,
                             'surface_m': bound_panels,
                             'merge_surface': True}
                    }
g1c['wing_l'] = {'symmetric': {'component': 'wing_r'}}
g1c['winglet_l'] = {'symmetric': {'component': 'winglet_r'}}
g1c['vertical_tail'] = {'workflow': ['create_structure', 'create_aero'],
                        'geometry': {'length': 2.5,
                                     'num_node': 11,
                                     'direction': [0., 0., 1.],
                                     'sweep': None,
                                     'dihedral': None},
                        'fem': {'stiffness_db': stiffness * sigma_tail,  # input tail stiffness
                                'mass_db': mass_tail,
                                'frame_of_reference_delta': [-1., 0., 0.]},
                        'aero': {'chord': [0.5, 0.5],
                                 'elastic_axis': 0.5,
                                 'surface_m': bound_panels}
                        }
g1c['horizontal_tail_right'] = {'workflow': ['create_structure', 'create_aero'],
                                'geometry': {'length': 2.5,
                                             'num_node': 11,
                                             'direction': [0., 1., 0.],
                                             'sweep': 0.,
                                             'dihedral': 0.},
                                'fem': {'stiffness_db': stiffness * sigma_tail,
                                        'mass_db': mass_tail,
                                        'frame_of_reference_delta': [-1, 0., 0.]},
                                'aero': {'chord': [0.5, 0.5],
                                         'elastic_axis': 0.4,
                                         'surface_m': bound_panels}
                                }
g1c['horizontal_tail_left'] = {'symmetric': {'component':'horizontal_tail_right'}}

print(type(g1c))
g1c_output = {i:g1c[i] for i in components}

print(type(g1c_output))
print(g1c_output)

<class 'list'>
<class 'dict'>
<class 'dict'>
{'fuselage': {'workflow': ['read_structure', 'read_aero'], 'geometry': {'length': 10, 'num_node': 11, 'direction': [1.0, 0.0, 0.0], 'sweep': 0.0, 'dihedral': 0.0}, 'fem': {'stiffness_db': array([[[15000000.,        0.,        0.,        0.,        0.,
                0.],
        [       0.,   100000.,        0.,        0.,        0.,
                0.],
        [       0.,        0.,   100000.,        0.,        0.,
                0.],
        [       0.,        0.,        0.,    22500.,        0.,
                0.],
        [       0.,        0.,        0.,        0.,    45000.,
                0.],
        [       0.,        0.,        0.,        0.,        0.,
           600000.]]]), 'mass_db': array([[[ 0.2 ,  0.  ,  0.  ,  0.  , -0.  ,  0.  ],
        [ 0.  ,  0.2 ,  0.  ,  0.  ,  0.  , -0.  ],
        [ 0.  ,  0.  ,  0.2 , -0.  ,  0.  ,  0.  ],
        [-0.  ,  0.  , -0.  ,  0.08,  0.  ,  0.  ],
        [-0.  , -0.  ,  0.  ,  0.  

Model settings built into a function

In [9]:
model_name = 'scratch_test'
model_route = 'scratch'

g1mm = {'model_name': model_name,
        'model_route': model_route,
                    # 'iterate_type': 'Full_Factorial',
                    # 'write_iterate_vars': True,
                    # 'iterate_vars': {'fuselage*geometry-length': np.linspace(7, 15., 3),
                    #                  'wing_r*geometry-length': np.linspace(15, 25., 3),
                    #                  'winglet_r*geometry-dihedral': np.pi / 180 * np.array([0, 20, 40])},
                    # 'iterate_labels': {'label_type': 'number',
                    #                    'print_name_var': 0},
        'assembly': {'include_aero': 1,
                     'default_settings': 1,  # beam_number and aero surface and
                     # surface_distribution
                     # selected by default one
                     # per component
                     'fuselage': {'upstream_component': '',
                                  'node_in_upstream': 0},
                     'wing_r': {'keep_aero_node': 1,
                                'upstream_component': 'fuselage',
                                'node_in_upstream': 0},
                     'winglet_r': {'keep_aero_node': 1,
                                   'upstream_component': 'wing_r',
                                   'node_in_upstream': 10},
                     'wing_l': {'upstream_component': 'fuselage',
                                'node_in_upstream': 0},
                     'winglet_l': {'upstream_component': 'wing_l',
                                   'node_in_upstream': 10},
                     'vertical_tail': {'upstream_component': 'fuselage',
                                       'node_in_upstream': 10},
                     'horizontal_tail_right': {'upstream_component': 'vertical_tail',
                                               'node_in_upstream': 10},
                     'horizontal_tail_left': {'upstream_component': 'vertical_tail',
                                              'node_in_upstream': 10}
                     }
        }
for ki in ['fuselage', 'wing_r', 'winglet_r',
           'wing_l', 'winglet_l', 'vertical_tail',
           'horizontal_tail_right', 'horizontal_tail_left']:

    if (ki not in ['include_aero', 'default_settings'] and
            ki not in components):
        del g1mm['assembly'][ki]

print(g1mm)

{'model_name': 'scratch_test', 'model_route': 'scratch', 'assembly': {'include_aero': 1, 'default_settings': 1, 'fuselage': {'upstream_component': '', 'node_in_upstream': 0}, 'wing_r': {'keep_aero_node': 1, 'upstream_component': 'fuselage', 'node_in_upstream': 0}, 'winglet_r': {'keep_aero_node': 1, 'upstream_component': 'wing_r', 'node_in_upstream': 10}, 'wing_l': {'upstream_component': 'fuselage', 'node_in_upstream': 0}, 'winglet_l': {'upstream_component': 'wing_l', 'node_in_upstream': 10}, 'vertical_tail': {'upstream_component': 'fuselage', 'node_in_upstream': 10}, 'horizontal_tail_right': {'upstream_component': 'vertical_tail', 'node_in_upstream': 10}, 'horizontal_tail_left': {'upstream_component': 'vertical_tail', 'node_in_upstream': 10}}}


This defines the model. The next step is to define a simulation, the generator model uses the nomenclature from Nastran
e.g:
* sol_112: Aeroelastic equilibrium
* sol_132: Modal solution
* sol_152: Flutter solution about arbitrary aeroelastic equilibrium

Lets try a modal solution


In [10]:
#############################################
# Modal solution                            #
#############################################
u_inf = 10
rho = 1.2
c_ref = 1.0
AoA = 0. * np.pi / 180
bound_panels = 8
sol_132 = {'sharpy': {'simulation_input': None,
                      'default_module': 'sharpy.routines.modal',
                      'default_solution': 'sol_132',
                      'default_solution_vars': {'num_modes': 10,
                                                'u_inf': u_inf,
                                                'rho': rho,
                                                'dt': c_ref / bound_panels / u_inf,
                                                'rotationA': [0., AoA, 0.],
                                                'panels_wake': 80,
                                                'horseshoe': True,
                                                'gravity_on': 0,
                                                'print_modal_matrices': False,
                                                'max_modal_disp': 0.15,
                                                'max_modal_rot_deg': 15.,
                                                'fsi_maxiter': 100,
                                                'fsi_tolerance': 1e-5,
                                                'fsi_relaxation': 0.1,
                                                'fsi_load_steps': 20,
                                                's_maxiter': 100,
                                                's_tolerance': 1e-5,
                                                's_relaxation': 1e-3,
                                                's_load_steps': 1,
                                                's_delta_curved': 1e-4,
                                                },
                      'default_sharpy': {},
                      'model_route': None
                      }
           }

This is everything that is required to run a sharpy simulation. Following the Object Oriented phylosophy, all the settings pertaining to a model are made into a class:

In [11]:
g1 = gm.Model('sharpy', ['sharpy'],
              model_dict=g1mm,

              components_dict=g1c,
              simulation_dict=sol_132)


# Now implement the same workflow using read_structure and read_areo

Note that read_structure and read_aero use h5utils.load_h5_in_dict(yyyy_file_handle) where yyyy is :
* fem
* aero

Already returns a dictionary with the required files

In [12]:
# Read the files directly
# Initialise a Sharpy_data class instance
workflow = {'read_structure','read_aero'}
ghale_sd = gm.Sharpy_data(workflow)

print(ghale_sd.structure_read)
print(ghale_sd.aero_read)

# Read structure
filename_s = 'simple_HALE.fem.h5'
ghale_sd.read_structure(filename_s)

# Read aero
filename_a = 'simple_HALE.aero.h5'
ghale_sd.read_aero(filename_a)

print(ghale_sd.fem)



1
1
{'app_forces': array([[0.  , 6.16, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  

Insert the simulation properties 

In [13]:
#####################################################################
# Run a flutter solution around an arbitrary aeroelastic equilibrium#
####################################################################
u_inf = 20
rho = 1.2
c_ref = 1.0
AoA = 0. * np.pi / 180
bound_panels = 8
sol_152 = {'sharpy': {'simulation_input': None,
                      'default_module': 'sharpy.routines.flutter',
                      'default_solution': 'sol_152',
                      'default_solution_vars': {
                          'flutter_reference': 21.,
                          'root_method': 'bisection',
                          'velocity_increment': 10.,
                          'flutter_error': 0.001,
                          'damping_tolerance': 5e-3,
                          'inout_coordinates': 'modes',
                          'secant_max_calls': 15,
                          'rho': rho,
                          'gravity_on': False,
                          'u_inf': u_inf,
                          'panels_wake': bound_panels * 10,
                          'dt': c_ref / bound_panels / u_inf,
                          'c_ref': c_ref,
                          'rom_method': '',
                          'rotationA': [0., AoA, 0.],
                          'horseshoe': True,
                          'num_modes': 20,
                          'fsi_maxiter': 100,
                          'fsi_tolerance': 1e-5,
                          'fsi_relaxation': 0.3,
                          'fsi_load_steps': 1,
                          's_maxiter': 100,
                          's_tolerance': 1e-5,
                          's_relaxation': 1e-3,
                          's_load_steps': 1,
                          's_delta_curved': 1e-4,
                          'add2_flow': [['StaticCoupled', 'plot']],
                      },
                      'default_sharpy': {},
                      'model_route': None}}
#############################################

In [14]:
try:
    print(x)
except:
    print('x not defined')
        

x not defined


In [15]:
model_route = os.getcwd() +'/simple_HALE'
print(model_route)

file2write = model_route
file_name = file2write+'.sharpy'
myTuple = ('John','Peter','Vicky')

x = '#'.join(myTuple)
print(x)
print(file_name)
y = '/'.join(file_name.split('/')[:-1])
print(y)
if os.path.exists(file_name):
    os.remove(file_name)
    print('filename exists and was removed')
elif not os.path.exists('/'.join(file_name.split('/')[:-1])):
   os.makedirs('/'.join(file_name.split('/')[:-1]))


/home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft/simple_HALE
John#Peter#Vicky
/home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft/simple_HALE.sharpy
/home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft
filename exists and was removed


In [16]:
# Leave the aero and fem files as is and change the model/case name to simple_HALE
# Need to instantiate a class of built_models
model_route = os.getcwd() +'/simple_HALE'

sharpyInstance = gm.Sharpy_data(workflow=['structure_input', 'aero_input'])

case_route = model_route
case_name = 'simple_HALE'
folder2write = case_route 
file2write = folder2write
gu.change_dic(sol_152['sharpy'], 'folder', folder2write)
sharpyInstance.sim = gm.Simulation(sim_type='sharpy',
                                          settings_sim=sol_152['sharpy'],
                                          case_route=folder2write,
                                          case_name= case_name)
sharpyInstance.sim.get_sharpy(
inp=sol_152['sharpy']['simulation_input'])
sharpyInstance.write_sim(file2write+'.sharpy')

# Run the simulation
dataX = sharpy.sharpy_main.main(['', file2write+'.sharpy'])

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/pablodfs/FYP/Projects-SHARPy/aeroelasticPMOR_Optimization/parametric_aircraft
SHARPy being run is in /home/pablodfs/sharpy
The branch being run is dev_rhea
The 

/home/pablodfs/anaconda3/envs/sharpy_env/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


	state-space model produced in form:
			h_{n+1} = A h_{n} + B u_{n}
			with:
	x_n = h_n + Bp u_n
			...done in 4.56 sec
Scaling UVLM system with reference time 0.050000s
Non-dimensional time step set (0.125000)
System scaled in 1.933026s
Generating an instance of LinearBeam
Warning, projecting system with damping onto undamped modes
Scaling beam according to reduced time...
	Setting the beam time step to (0.1250)
Updating C and K matrices and natural frequencies with new normalised time...
Aeroelastic system assembled:
	Aerodynamic states: 728
	Structural states: 40
	Total states: 768
	Inputs: 64
	Outputs: 60
Final system is:
State-space system
States: 768
Inputs: 64
Outputs: 60

Generating an instance of DynamicLoads
Variable print_info has no assigned value in the settings file.
    will default to the value: True
Variable secant_max_calls has no assigned value in the settings file.
    will default to the value: 0
Variable mach_number has no assigned value in the settings file.
    

These dictionaries can also be obtained by following the class Model and letting the inbuilt class functions deal with it. For that, there needs to be some settings defined:


In [19]:
settings = {'workflow':{'read_structure','read_aero'}
           }

## Testing how the stiffness db matrix is built 

In [27]:
base_stiffness_main = sigma * np.diag([ea, ga, ga, gj, eiy, eiz])
base_stiffness_fuselage = base_stiffness_main.copy()*sigma_fuselage
base_stiffness_fuselage[4, 4] = base_stiffness_fuselage[5, 5]
base_stiffness_tail = base_stiffness_main.copy()*sigma_tail
base_stiffness_tail[4,4]=base_stiffness_tail[5,5]

print(base_stiffness_main)
stiffness = np.zeros((3,6,6))
print(stiffness)
stiffness[0, ...] = base_stiffness_main
print('\n\n')
print(stiffness)
stiffness[1, ...] = base_stiffness_fuselage
print('\n\n')
print(stiffness)
stiffness[2, ...] = base_stiffness_tail


[[22500000.        0.        0.        0.        0.        0.]
 [       0.   150000.        0.        0.        0.        0.]
 [       0.        0.   150000.        0.        0.        0.]
 [       0.        0.        0.    22500.        0.        0.]
 [       0.        0.        0.        0.    45000.        0.]
 [       0.        0.        0.        0.        0.   900000.]]
[[[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0.]]]



[[[22500000.        0.        0.        0.        0.        0.]
  [       0.   150000.        0.        0.        0.        0.]
  [       0.        0.   150000.        0.        0.        0.]
  [       0.        0.  